In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# read data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')

# check data

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print('train data shape is:',train.shape)
cols_with_missing = [col for col in train.columns if train[col].isnull().any()]
print('null columns in train data is : ',cols_with_missing)
s = (train.dtypes == 'object')
print('train Categorical variables:',list(s[s].index))

print()

print('test data shape is:',test.shape)
cols_with_missing = [col for col in test.columns if test[col].isnull().any()]
print('null columns in test data is : ',cols_with_missing)
s_ = (test.dtypes == 'object')
print('test Categorical variables:',list(s_[s_].index))

**there are no missing value and f_27 is only object column**

In [ ]:
print(train['target'].value_counts())
print()
label = train.groupby('target').sum()
label = label.index

plt.figure(figsize=(4,4))
plt.pie(train['target'].value_counts(),counterclock=False, startangle=90,labels=label,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

**target rate is almost same**

In [ ]:
train_=pd.DataFrame(train['f_27'].value_counts())
train_['count']=1
train_=train_.groupby('f_27').sum()
train_['rate']=train_['count']/741354

plt.figure(figsize=(4,4))
plt.pie(train_['count'],counterclock=False, startangle=90, pctdistance=0.7)
plt.show()

train_.T

**f_27 column almost 85% data is different each other**

In [ ]:
train.describe(include='all').T

In [ ]:
cont_features = train.columns.drop('f_27')
target_1=train[train['target']==1]
target_0=train[train['target']==0]

print("Feature distribution of important columns : ")
ncols = 3
nrows = 10

fig, axes = plt.subplots(nrows, ncols, figsize=(40,50), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = cont_features[r*ncols+c+1]
        sns.kdeplot(x=target_1[col], ax=axes[r, c], color='red', label='target label 1')
        sns.kdeplot(x=target_0[col], ax=axes[r, c], color='blue', label='target label 0')
        sns.kdeplot(x=test[col], ax=axes[r, c], color='green', label='test data')
        axes[r, c].set_ylabel('')
        axes[r, c].set_xlabel(col, fontsize=15, fontweight='bold')
        axes[r, c].tick_params(labelsize=10, width=0.5)
        axes[r, c].xaxis.offsetText.set_fontsize(20)
        axes[r, c].yaxis.offsetText.set_fontsize(20)
plt.show()

**f_07~f_18 and f_29,f_30 is looks like categorical colum**

In [ ]:
categorical_data=train.dtypes == 'int'

train_int_col=list(categorical_data[categorical_data].index)
train_int_col

**those colum have integer data types**

# create model most simple type

In [ ]:
def fit_data(X,y,clf):
    #Separate data into training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=0)
    
    clf.fit(X_train, y_train)
    
    y_pred=np.round(clf.predict(X_valid))
    accuracy=accuracy_score(y_pred, y_valid)
    print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_valid, y_pred)))
    
    plt.figure(figsize=(10,13))
    importances = pd.Series(clf.feature_importances_, index = X.columns)
    importances = importances.sort_values()
    importances.plot(kind = "barh")
    plt.title("importance in the LightGBM Model")
    plt.show()
    
    return clf

In [ ]:
X=train.drop(['id','f_27','target'],axis = 1)
X=X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
y = train.target
clf = lgb.LGBMClassifier()
clf=fit_data(X,y,clf)

# add some statistics feature from train data 

In [ ]:
train_statics=pd.DataFrame()
train_statics['origin_std']=train.drop(train_int_col,axis=1).std(axis=1)
train['origin_std']=train_statics['origin_std']

# add f_27 column feature and train again

In [ ]:
train_object_data=pd.DataFrame()

for i in range(10):
    train_object_data['label_'+str(i)]=train['f_27'].str[i:i+1]

train_object_data

In [ ]:
col_name_list=train_object_data.columns

In [ ]:
for i in range(10): 
    print('label_'+str(i))
    print(train_object_data[col_name_list[i]].value_counts())
    
    label = train_object_data.groupby(col_name_list[i]).sum()
    label = label.index

    plt.figure(figsize=(5,5))
    plt.pie(train_object_data[col_name_list[i]].value_counts(),counterclock=False, startangle=90,labels=label,
           autopct='%1.1f%%', pctdistance=0.7)
    plt.show()

In [ ]:
df_spel_count=pd.DataFrame()
spel_list=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']
for i in range(20):
    df_spel_count[spel_list[i]]=train_object_data[train_object_data==spel_list[i]].count(axis=1)

In [ ]:
df_spel_count_=pd.DataFrame()
df_spel_count_['sum']=df_spel_count.sum(axis=1)
df_spel_count_['std']=df_spel_count.std(axis=1)

df_spel_count['sum_2']=df_spel_count['A']+df_spel_count['B']
df_spel_count['diff_2']=df_spel_count['A']-df_spel_count['B']
df_spel_count['rate_2']=df_spel_count['A']/df_spel_count['B']
df_spel_count['sum_4']=df_spel_count['A']+df_spel_count['B']+df_spel_count['C']+df_spel_count['D']

df_spel_count['sum']=df_spel_count_['sum']
df_spel_count['std']=df_spel_count_['std']

df_spel_count

In [ ]:
ordinal_encoder = OrdinalEncoder()
train_object_data[train_object_data.columns] = ordinal_encoder.fit_transform(train_object_data)
train_object_data['cat_sum']=train_object_data.drop(['label_0','label_2','label_5'],axis=1).sum(axis=1)
train_object_data['cat_std']=train_object_data.drop(['label_0','label_2','label_5','cat_sum'],axis=1).std(axis=1)
train_object_data

In [ ]:
X=pd.concat([train.drop(['id','f_27','target'],axis = 1),df_spel_count,train_object_data],axis=1)
X=X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
clf = lgb.LGBMClassifier()
clf=fit_data(X,y,clf)

# change some colum dtypes and train again

In [ ]:
label_list=['label_0','label_2','label_5']
train_object_data.iloc[:,0:10]=train_object_data.iloc[:,0:10].astype('int')
train_object_data[label_list]=train_object_data[label_list].astype('category')
train[train_int_col[1:15]]=train[train_int_col[1:15]].astype('category')
print(train_object_data.dtypes)
print(train.dtypes)

In [ ]:
X=pd.concat([train.drop(['id','f_27','target'],axis = 1),df_spel_count,train_object_data],axis=1)
X=X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
clf = lgb.LGBMClassifier()
clf=fit_data(X,y,clf)

# make more feature with LGBMRegressor and PCA

In [ ]:
regressor_1 = lgb.LGBMRegressor()
regressor_1=fit_data(train_object_data.drop(['label_0','label_2','label_5','cat_std','cat_sum'],axis=1),y,regressor_1)
train_object_data['regressor_object_data']=regressor_1.predict(train_object_data.drop(['label_0','label_2','label_5','cat_std','cat_sum'],axis=1))

In [ ]:
regressor_1_ = lgb.LGBMRegressor()
regressor_2_ = lgb.LGBMRegressor()
regressor_1_=fit_data(df_spel_count.drop(['sum_2','sum_4','diff_2','rate_2','sum','std'],axis=1),y,regressor_1_)
df_spel_count['regressor_spel_count']=regressor_1_.predict(df_spel_count.drop(['sum_2','sum_4','diff_2','rate_2','sum','std'],axis=1))
regressor_2_=fit_data(df_spel_count.drop(['sum_2','sum_4','diff_2','rate_2','sum','std'],axis=1),y,regressor_2_)
df_spel_count['regressor_spel_count']=regressor_2_.predict(df_spel_count.drop(['sum_2','sum_4','diff_2','rate_2','sum','std'],axis=1))

In [ ]:
from sklearn import preprocessing
mm = preprocessing.MinMaxScaler()
l_2d_min_max = mm.fit_transform(train_object_data.drop(['cat_sum','cat_std'],axis=1))

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(l_2d_min_max)

feature = pca.transform(l_2d_min_max)

train_object_data['pca_0']=feature[:,0]
train_object_data['pca_1']=feature[:,1]
train_object_data['pca_2']=feature[:,2]

In [ ]:
X=pd.concat([train.drop(['id','f_27','target'],axis = 1),df_spel_count,
             train_object_data],axis=1)
X=X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
clf = lgb.LGBMClassifier()
clf=fit_data(X,y,clf)

# set some parameter of LGBMClassifierModel and train again

In [ ]:
clf = lgb.LGBMClassifier(num_leaves= 200,
                         silent=True,  
                         n_jobs=10,
                         n_estimators=10000,
                         learning_rate=0.08)
clf=fit_data(X,y,clf)
#0.9544

# drop some columns from training data

In [ ]:
X=pd.concat([train.drop(['id','f_27','target'],axis = 1),df_spel_count.drop(['S','T','sum'],axis = 1),
             train_object_data],axis=1)
X=X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
clf = lgb.LGBMClassifier(num_leaves= 200,
                         silent=True,  
                         n_jobs=10,
                         n_estimators=10000,
                         learning_rate=0.08)
clf=fit_data(X,y,clf)

# train again with all data

In [ ]:
clf.fit(X, y)

# prepare test data for submit

In [ ]:
test_object_data=pd.DataFrame()
train_int_col.pop(-1)
test_statics=pd.DataFrame()
test_statics['origin_std']=test.drop(train_int_col,axis=1).std(axis=1)
test['origin_std']=test_statics['origin_std']

for i in range(10):
    test_object_data['label_'+str(i)]=test['f_27'].str[i:i+1]

df_spel_count=pd.DataFrame()
for i in range(20):
    df_spel_count[spel_list[i]]=test_object_data[test_object_data==spel_list[i]].count(axis=1)
    
df_spel_count_=pd.DataFrame()
df_spel_count_['sum']=df_spel_count.sum(axis=1)
df_spel_count_['std']=df_spel_count.std(axis=1)

df_spel_count['sum_2']=df_spel_count['A']+df_spel_count['B']
df_spel_count['diff_2']=df_spel_count['A']-df_spel_count['B']
df_spel_count['rate_2']=df_spel_count['A']/df_spel_count['B']
df_spel_count['sum_4']=df_spel_count['A']+df_spel_count['B']+df_spel_count['C']+df_spel_count['D']

df_spel_count['sum']=df_spel_count_['sum']
df_spel_count['std']=df_spel_count_['std']

ordinal_encoder = OrdinalEncoder()
test_object_data[test_object_data.columns] = ordinal_encoder.fit_transform(test_object_data)
test_object_data['cat_sum']=test_object_data.drop(['label_0','label_2','label_5'],axis=1).sum(axis=1)
test_object_data['cat_std']=test_object_data.drop(['label_0','label_2','label_5','cat_sum'],axis=1).std(axis=1)

test_object_data.iloc[:,0:10]=test_object_data.iloc[:,0:10].astype('int')
test_object_data[label_list]=test_object_data[label_list].astype('category')
test[train_int_col[1:15]]=test[train_int_col[1:15]].astype('category')

test_object_data['regressor_object_data']=regressor_1.predict(test_object_data.drop(['label_0','label_2','label_5','cat_std','cat_sum'],axis=1))
df_spel_count['regressor_spel_count']=regressor_1_.predict(df_spel_count.drop(['sum_2','sum_4','diff_2','rate_2','sum','std'],axis=1))
df_spel_count['regressor_spel_count']=regressor_2_.predict(df_spel_count.drop(['sum_2','sum_4','diff_2','rate_2','sum','std'],axis=1))

mm = preprocessing.MinMaxScaler()
l_2d_min_max = mm.fit_transform(test_object_data.drop(['cat_sum','cat_std'],axis=1))
feature = pca.transform(l_2d_min_max)
test_object_data['pca_0']=feature[:,0]
test_object_data['pca_1']=feature[:,1]
test_object_data['pca_2']=feature[:,2]

test_X=pd.concat([test.drop(['id','f_27'],axis = 1),df_spel_count.drop(['S','T','sum'],axis = 1),
                  test_object_data],axis=1)
test_X=test_X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [ ]:
print('train data shape is : ',X.shape)
print()
print('test data shape is : ',test_X.shape)

# make prediction for submit

In [ ]:
submission['target']=clf.predict(test_X)

print(submission['target'].value_counts())
print()
label = submission.groupby('target').sum()
label = label.index

plt.figure(figsize=(3,3))
plt.pie(submission['target'].value_counts(),counterclock=False, startangle=90,labels=label,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

**prediction "target" data looks like train data**

In [ ]:
submission.to_csv('submission.csv', index=False)